# Historical Prices

In [4]:
def get_stock_history_intraday(symbol="AAPL", from_yyyy_mm_dd = "all", to_yyyy_mm_dd = "all", api_token = config['api_key']):
    
    final_df = pd.DataFrame()
    
    days_to_cover = 400

    temp_from_date = from_yyyy_mm_dd

    while days_to_cover >= 0:

        temp_to_date = temp_from_date + DT.timedelta(days=100)

        if temp_to_date >= pd.to_datetime(SPLIT_DATE):
            temp_to_date = pd.to_datetime(SPLIT_DATE) + DT.timedelta(days=1)
            days_to_cover = -1

        from_dt_timestamp = int(datetime.timestamp(temp_from_date))
        to_dt_timestamp = int(datetime.timestamp(temp_to_date))
    
        url = f"https://eodhistoricaldata.com/api/intraday/{symbol}.US?api_token={config['api_key']}&interval=1m&from={from_dt_timestamp}&to={to_dt_timestamp}"
    
        r = hfs.requests_retry_session().get(url)
        c = 0
        #print(r.status_code)
        while r.status_code != requests.codes.ok:

            if r.status_code == 429: 
                #print("waiting 5 sec for ", symbol)
                if c == 5:
                    print("c == 5 breaking", symbol)
                    break
                time.sleep(random.randint(3, 6))
                try:
                    r = hfs.requests_retry_session().get(url)
                    c+=1
                except:
                    continue

            else:
                print(symbol, " breaking : " + str(r.status_code) + " <> status code") 
                break
                #return pd.DataFrame([{'Ticker' : symbol,  'Open' : r.reason,  'Date' : r.status_code,  'Close': url}]).reset_index(drop=True)
                print("******* raising Exception *****",type(r.status_code), r.status_code)
                raise Exception(symbol, r.status_code, r.reason, url) 

        else:

            df = pd.read_csv(StringIO(r.text), skipfooter=0, engine='python').reset_index(drop=True) # 
            df['Ticker'] = symbol
            
            final_df = pd.concat([final_df,df]).reset_index(drop=True)
            
        
        temp_from_date = temp_to_date
        days_to_cover -=100
    
    return final_df.reset_index(drop=True) 


In [5]:
# opening time = est 4 hrs = utc 8 hrs hrs
# closing time = est 19:55 hrs = utc 23:55 hrs


In [6]:
files_in_PROCESSING_TEMP = os.listdir(f"{PROJ_PATH}{PROCESSING_TEMP_FOLDER_PATH}") 
files_in_PROCESSING_TEMP

all_stocks_ohlcv = pd.read_pickle( f"{PROJ_PATH}{ML_DATA_FOLDER_PATH}{[x for x in files_in_ML_DATA if 'all_stocks_ohlcv' in x][0]}") 
print("#Tickers: ", all_stocks_ohlcv['Ticker'].nunique())
print(all_stocks_ohlcv.shape)
all_stocks_ohlcv.tail(2) 

#Tickers:  776
(435218, 10)


,company_Name,Ticker,Date,Adjusted_Open,Adjusted_High,Adjusted_Low,Adjusted_close,Adjusted_Volume,Exchange,close_to_adj_close_ratio
435216,ZTO Express (Cayman) Inc,ZTO,2023-07-10,24.76,25.130,24.69,25.10,2082500.0,NYSE,1.0
435217,ZTO Express (Cayman) Inc,ZTO,2023-07-11,25.22,26.365,25.10,26.25,4233750.0,NYSE,1.0


### Getting IntraDay OHLCV

In [7]:
focus_tickers = [x for x in all_stocks_ohlcv['Ticker'].unique()]
len(focus_tickers) 

776

In [8]:
stocks_daily_ohlcv = pd.DataFrame()

Hourly_FROM_date = pd.to_datetime(date.today(), format='%Y-%m-%d') - DT.timedelta(days=400) 
Hourly_FROM_date = Hourly_FROM_date.replace(minute=0, hour=8, second=0, year=Hourly_FROM_date.date().year, month=Hourly_FROM_date.date().month, day=Hourly_FROM_date.date().day)
print(Hourly_FROM_date)

Hourly_TO_date =  pd.to_datetime(SPLIT_DATE)
Hourly_TO_date = Hourly_TO_date.replace(minute=55, hour=23, second=0, year=Hourly_TO_date.date().year, month=Hourly_TO_date.date().month, day=Hourly_TO_date.date().day)
print(Hourly_TO_date)

tickers = focus_tickers
print("total #tickers: ", len(tickers))

max_tickers_lot_size = 82

for i in range(0 , int(len(tickers)/max_tickers_lot_size) + 1):
    start_list = i * max_tickers_lot_size
    end_list = start_list + max_tickers_lot_size
    
    print(f'sending lot numbr {i} with {len(tickers[start_list: end_list])} tickers')
    
    args = ((symbol,      
            Hourly_FROM_date,   # From date
            Hourly_TO_date,
            config['api_key']) for symbol in tickers[start_list: end_list] )

    temp_stocks_daily_ohlcv = hfs.get_data_from_API(tickers = tickers[start_list: end_list],
                                                      func_to_run = get_stock_history_intraday,
                                                      func_args = args ) 
    
    stocks_daily_ohlcv = pd.concat([stocks_daily_ohlcv, temp_stocks_daily_ohlcv]).reset_index(drop=True)

print("unique #tickers: ", stocks_daily_ohlcv['Ticker'].nunique()) 
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(2) 


2022-06-06 08:00:00
2023-07-11 23:55:00
total #tickers:  776
sending lot numbr 0 with 82 tickers
chunk  1 :  82
total time:  69.12559413909912
returning df
sending lot numbr 1 with 82 tickers
chunk  1 :  82
total time:  61.31227374076843
returning df
sending lot numbr 2 with 82 tickers
chunk  1 :  82
total time:  56.507558822631836
returning df
sending lot numbr 3 with 82 tickers
chunk  1 :  82
total time:  60.06504821777344
returning df
sending lot numbr 4 with 82 tickers
chunk  1 :  82
total time:  55.02122712135315
returning df
sending lot numbr 5 with 82 tickers
chunk  1 :  82
total time:  63.079341173172
returning df
sending lot numbr 6 with 82 tickers
chunk  1 :  82
total time:  57.17530179023743
returning df
sending lot numbr 7 with 82 tickers
chunk  1 :  82
total time:  60.711326122283936
returning df
sending lot numbr 8 with 82 tickers
chunk  1 :  82
total time:  89.33051800727844
returning df
sending lot numbr 9 with 38 tickers
chunk  1 :  38
total time:  32.655242919921875
r

,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,Ticker,Value
74071970,1.689030e+09,0.0,2023-07-10 23:00:00,25.10,25.10,25.10,25.10,273173.0,ZTO,NaN
74071971,1.689033e+09,0.0,2023-07-10 23:52:00,25.07,25.07,25.07,25.07,40.0,ZTO,NaN


In [9]:
stocks_daily_ohlcv.head(2) 

,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,Ticker,Value
0,1.654528e+09,0.0,2022-06-06 15:00:00,63.265,63.30,63.230,63.2806,9261.0,AA,NaN
1,1.654528e+09,0.0,2022-06-06 15:01:00,63.310,63.42,63.305,63.4100,15483.0,AA,NaN


### Preparing minute ohlcv df

In [10]:
stocks_daily_ohlcv.drop(columns=['Timestamp','Gmtoffset'], inplace=True) 

In [11]:
stocks_daily_ohlcv['Datetime'] = pd.to_datetime(stocks_daily_ohlcv['Datetime'])
stocks_daily_ohlcv = stocks_daily_ohlcv.sort_values(by=['Ticker','Datetime']).reset_index(drop=True)
stocks_daily_ohlcv['Datetime'] = stocks_daily_ohlcv['Datetime'].dt.tz_localize('UTC')
stocks_daily_ohlcv['Datetime'] = stocks_daily_ohlcv['Datetime'].dt.tz_convert('US/Eastern')
stocks_daily_ohlcv['Date'] = stocks_daily_ohlcv['Datetime'].dt.date
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.head(2) 


(74071972, 9)


,Datetime,Open,High,Low,Close,Volume,Ticker,Value,Date
0,2022-06-06 11:00:00-04:00,63.265,63.30,63.230,63.2806,9261.0,AA,NaN,2022-06-06
1,2022-06-06 11:01:00-04:00,63.310,63.42,63.305,63.4100,15483.0,AA,NaN,2022-06-06


In [12]:
stocks_daily_ohlcv.tail(2) 

,Datetime,Open,High,Low,Close,Volume,Ticker,Value,Date
74071970,2023-07-10 19:00:00-04:00,25.10,25.10,25.10,25.10,273173.0,ZTO,NaN,2023-07-10
74071971,2023-07-10 19:52:00-04:00,25.07,25.07,25.07,25.07,40.0,ZTO,NaN,2023-07-10


In [13]:
stocks_daily_ohlcv['Date'].max()

datetime.date(2023, 7, 11)

In [14]:
stocks_daily_ohlcv['time'] = stocks_daily_ohlcv['Datetime'].dt.time 

In [15]:
hr_cat_conditions = [(stocks_daily_ohlcv['Datetime'].dt.time>=DT.time(4, 0)) & (stocks_daily_ohlcv['Datetime'].dt.time<DT.time(9, 30)),
                     
                     (stocks_daily_ohlcv['Datetime'].dt.time>=DT.time(9, 30)) & (stocks_daily_ohlcv['Datetime'].dt.time<DT.time(9, 35)),
                     (stocks_daily_ohlcv['Datetime'].dt.time>=DT.time(9, 35)) & (stocks_daily_ohlcv['Datetime'].dt.time<DT.time(9, 40)),
                     (stocks_daily_ohlcv['Datetime'].dt.time>=DT.time(9, 40)) & (stocks_daily_ohlcv['Datetime'].dt.time<DT.time(9, 45)),
                     (stocks_daily_ohlcv['Datetime'].dt.time>=DT.time(9, 45)) & (stocks_daily_ohlcv['Datetime'].dt.time<DT.time(10,0)),
                     (stocks_daily_ohlcv['Datetime'].dt.time>=DT.time(10, 0)) & (stocks_daily_ohlcv['Datetime'].dt.time<DT.time(15, 30)),
                     
                      (stocks_daily_ohlcv['Datetime'].dt.time>=DT.time(15, 30)) & (stocks_daily_ohlcv['Datetime'].dt.time<DT.time(16, 0)),
                      
                      (stocks_daily_ohlcv['Datetime'].dt.time>=DT.time(16, 0)) & (stocks_daily_ohlcv['Datetime'].dt.time<=DT.time(19, 59)) 
                     ]

hr_cat_choices = ['Pre_market',
                  'mkt_op_9_30_to_9_35', 
                  'mkt_op_9_35_to_9_40', 
                  'mkt_op_9_40_to_9_45', 
                  'mkt_op_9_45_to_10', 
                  'mkt_10_to_3_30', 
                  'mkt_cl_3_30_to_4',
                  'After_market']

stocks_daily_ohlcv['time_cat'] = np.select(hr_cat_conditions, hr_cat_choices, default= "unknown" )
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.head(2) 


(74071972, 11)


,Datetime,Open,High,Low,Close,Volume,Ticker,Value,Date,time,time_cat
0,2022-06-06 11:00:00-04:00,63.265,63.30,63.230,63.2806,9261.0,AA,NaN,2022-06-06,11:00:00,mkt_10_to_3_30
1,2022-06-06 11:01:00-04:00,63.310,63.42,63.305,63.4100,15483.0,AA,NaN,2022-06-06,11:01:00,mkt_10_to_3_30


In [16]:
stocks_daily_ohlcv['time_cat'].value_counts() 

mkt_10_to_3_30         55678901
mkt_cl_3_30_to_4        5535396
Pre_market              4422647
After_market            3205126
mkt_op_9_45_to_10       2598346
mkt_op_9_30_to_9_35      887197
mkt_op_9_40_to_9_45      869333
mkt_op_9_35_to_9_40      869261
unknown                    5765
Name: time_cat, dtype: int64

In [18]:
stocks_daily_ohlcv['avg_price'] = (stocks_daily_ohlcv['Open'] + stocks_daily_ohlcv['High'] + stocks_daily_ohlcv['Low'] + stocks_daily_ohlcv['Close']) / 4
stocks_daily_ohlcv['vol_price'] = stocks_daily_ohlcv['avg_price'] * stocks_daily_ohlcv['Volume']
stocks_daily_ohlcv.tail(2) 

,Datetime,Open,High,Low,Close,Volume,Ticker,Value,Date,time,time_cat,avg_price,vol_price
74071970,2023-07-10 19:00:00-04:00,25.10,25.10,25.10,25.10,273173.0,ZTO,NaN,2023-07-10,19:00:00,After_market,25.10,6856642.3
74071971,2023-07-10 19:52:00-04:00,25.07,25.07,25.07,25.07,40.0,ZTO,NaN,2023-07-10,19:52:00,After_market,25.07,1002.8


### Groupby Hour Category df

In [19]:
stocks_d_hlv_cat = stocks_daily_ohlcv.groupby(['Ticker','Date','time_cat']).agg({
    "High" : "max",
    "Low" :  "min",
    "Volume" : "sum",
    "vol_price" : "sum"
}).reset_index()

stocks_d_hlv_cat['time_cat_avg_price'] = stocks_d_hlv_cat['vol_price'] / stocks_d_hlv_cat['Volume']
del stocks_d_hlv_cat['vol_price']
print(stocks_d_hlv_cat.shape)
stocks_d_hlv_cat.head(2) 


(1565607, 7)


,Ticker,Date,time_cat,High,Low,Volume,time_cat_avg_price
0,AA,2022-06-06,After_market,64.10,63.8,151955.0,64.018133
1,AA,2022-06-06,mkt_10_to_3_30,64.25,62.4,2330464.0,63.313121


In [20]:
dates_df = pd.DataFrame()
dates_df['Date'] = sorted(stocks_d_hlv_cat['Date'].unique())
dates_df['Key'] = 1

Tickers_df = pd.DataFrame()
Tickers_df['Ticker'] = sorted(stocks_d_hlv_cat['Ticker'].unique())
Tickers_df['Key'] = 1

hour_cat_df = pd.DataFrame()
hour_cat_df['time_cat'] = sorted(stocks_d_hlv_cat['time_cat'].unique())
hour_cat_df['Key'] = 1

dates_tickers_df  = pd.merge(dates_df, Tickers_df, on ='Key')
print(dates_tickers_df.shape)
cross_joined_df  = pd.merge(dates_tickers_df, hour_cat_df, on ='Key').drop("Key", 1)
print(cross_joined_df.shape)

print(dates_df.shape[0] * Tickers_df.shape[0] * hour_cat_df.shape[0] )

cross_joined_df.tail()

(213125, 3)
(1918125, 3)
1918125


,Date,Ticker,time_cat
1918120,2023-07-11,ZTO,mkt_op_9_30_to_9_35
1918121,2023-07-11,ZTO,mkt_op_9_35_to_9_40
1918122,2023-07-11,ZTO,mkt_op_9_40_to_9_45
1918123,2023-07-11,ZTO,mkt_op_9_45_to_10
1918124,2023-07-11,ZTO,unknown


In [21]:
print(stocks_d_hlv_cat.shape)
stocks_d_hlv_cat  = pd.merge(cross_joined_df, stocks_d_hlv_cat, how='left', 
                            on = ['Date','Ticker','time_cat'])
print(stocks_d_hlv_cat.shape)
stocks_d_hlv_cat.tail(2)



(1565607, 7)
(1918125, 7)


,Date,Ticker,time_cat,High,Low,Volume,time_cat_avg_price
1918123,2023-07-11,ZTO,mkt_op_9_45_to_10,NaN,NaN,NaN,NaN
1918124,2023-07-11,ZTO,unknown,NaN,NaN,NaN,NaN


In [22]:
time_cat_seq_dict = {'Pre_market' : 1, 
                    'mkt_op_9_30_to_9_35' : 2,
                    'mkt_op_9_35_to_9_40' : 3, 
                    'mkt_op_9_40_to_9_45' : 4,
                    'mkt_op_9_45_to_10' : 5,
                    'mkt_10_to_3_30' : 6,
                    'mkt_cl_3_30_to_4' : 7, 
                    'After_market' : 8
                    }

stocks_d_hlv_cat['time_cat_seq'] = stocks_d_hlv_cat['time_cat'].map(time_cat_seq_dict)
stocks_d_hlv_cat.tail() 

,Date,Ticker,time_cat,High,Low,Volume,time_cat_avg_price,time_cat_seq
1918120,2023-07-11,ZTO,mkt_op_9_30_to_9_35,NaN,NaN,NaN,NaN,2.0
1918121,2023-07-11,ZTO,mkt_op_9_35_to_9_40,NaN,NaN,NaN,NaN,3.0
1918122,2023-07-11,ZTO,mkt_op_9_40_to_9_45,NaN,NaN,NaN,NaN,4.0
1918123,2023-07-11,ZTO,mkt_op_9_45_to_10,NaN,NaN,NaN,NaN,5.0
1918124,2023-07-11,ZTO,unknown,NaN,NaN,NaN,NaN,NaN


In [23]:
stocks_d_hlv_cat = stocks_d_hlv_cat.sort_values(by=['Ticker','Date','time_cat_seq'], ascending=[True,True,True]).reset_index(drop=True)
print(stocks_d_hlv_cat.shape)
stocks_d_hlv_cat.tail()


(1918125, 8)


,Date,Ticker,time_cat,High,Low,Volume,time_cat_avg_price,time_cat_seq
1918120,2023-07-11,ZTO,mkt_op_9_45_to_10,NaN,NaN,NaN,NaN,5.0
1918121,2023-07-11,ZTO,mkt_10_to_3_30,NaN,NaN,NaN,NaN,6.0
1918122,2023-07-11,ZTO,mkt_cl_3_30_to_4,NaN,NaN,NaN,NaN,7.0
1918123,2023-07-11,ZTO,After_market,NaN,NaN,NaN,NaN,8.0
1918124,2023-07-11,ZTO,unknown,NaN,NaN,NaN,NaN,NaN


In [24]:
for col in ['High', 'Low', 'time_cat_avg_price']:
    stocks_d_hlv_cat[col] = stocks_d_hlv_cat.groupby(['Ticker'])[col].transform(lambda x: x.ffill() )
    stocks_d_hlv_cat[col] = stocks_d_hlv_cat.groupby(['Ticker'])[col].transform(lambda x: x.bfill() )
stocks_d_hlv_cat['Volume'].fillna(0, inplace=True) 

In [25]:
stocks_d_hlv_cat.isna().sum() 

Date                       0
Ticker                     0
time_cat                   0
High                       0
Low                        0
Volume                     0
time_cat_avg_price         0
time_cat_seq          213125
dtype: int64

In [26]:
stocks_d_hlv_cat.to_pickle(f'{PROJ_PATH}{PROCESSING_TEMP_FOLDER_PATH}stocks_intraday_timecats_research.pkl') 
print(stocks_d_hlv_cat.shape)
stocks_d_hlv_cat.head(2) 


(1918125, 8)


,Date,Ticker,time_cat,High,Low,Volume,time_cat_avg_price,time_cat_seq
0,2022-06-06,AA,Pre_market,64.25,62.4,0.0,63.313121,1.0
1,2022-06-06,AA,mkt_op_9_30_to_9_35,64.25,62.4,0.0,63.313121,2.0


In [27]:
#stocks_d_hlv_cat.columns = [ 'Date', 'Ticker', 'time_cat', 'High', 'Low', 'Volume', 'time_cat_avg_price','time_cat_seq']
print(stocks_d_hlv_cat.shape)
stocks_d_hlv_cat_pivot = stocks_d_hlv_cat.pivot_table(index= [ 'Date', 'Ticker'], columns='time_cat', values= ['High','Low','Volume','time_cat_avg_price']).reset_index()
print(stocks_d_hlv_cat_pivot.shape)
stocks_d_hlv_cat_pivot.columns = [x for x in stocks_d_hlv_cat_pivot.columns.map('__'.join).str.strip('__')]
print(stocks_d_hlv_cat_pivot.shape)
stocks_d_hlv_cat_pivot.tail(2) 

(1918125, 8)
(213125, 38)
(213125, 38)


,Date,Ticker,High__After_market,High__Pre_market,High__mkt_10_to_3_30,High__mkt_cl_3_30_to_4,High__mkt_op_9_30_to_9_35,High__mkt_op_9_35_to_9_40,High__mkt_op_9_40_to_9_45,High__mkt_op_9_45_to_10,High__unknown,Low__After_market,Low__Pre_market,Low__mkt_10_to_3_30,Low__mkt_cl_3_30_to_4,Low__mkt_op_9_30_to_9_35,Low__mkt_op_9_35_to_9_40,Low__mkt_op_9_40_to_9_45,Low__mkt_op_9_45_to_10,Low__unknown,Volume__After_market,Volume__Pre_market,Volume__mkt_10_to_3_30,Volume__mkt_cl_3_30_to_4,Volume__mkt_op_9_30_to_9_35,Volume__mkt_op_9_35_to_9_40,Volume__mkt_op_9_40_to_9_45,Volume__mkt_op_9_45_to_10,Volume__unknown,time_cat_avg_price__After_market,time_cat_avg_price__Pre_market,time_cat_avg_price__mkt_10_to_3_30,time_cat_avg_price__mkt_cl_3_30_to_4,time_cat_avg_price__mkt_op_9_30_to_9_35,time_cat_avg_price__mkt_op_9_35_to_9_40,time_cat_avg_price__mkt_op_9_40_to_9_45,time_cat_avg_price__mkt_op_9_45_to_10,time_cat_avg_price__unknown
213123,2023-07-11,ZM,67.85,67.85,67.85,67.85,67.85,67.85,67.85,67.85,67.85,67.0000,67.0000,67.0000,67.0000,67.0000,67.0000,67.0000,67.0000,67.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.593417,67.593417,67.593417,67.593417,67.593417,67.593417,67.593417,67.593417,67.593417
213124,2023-07-11,ZTO,25.10,25.10,25.10,25.10,25.10,25.10,25.10,25.10,25.10,24.7315,24.7315,24.7315,24.7315,24.7315,24.7315,24.7315,24.7315,24.7315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.098907,25.098907,25.098907,25.098907,25.098907,25.098907,25.098907,25.098907,25.098907


In [28]:
stocks_d_hlv_cat_pivot.columns

Index(['Date', 'Ticker', 'High__After_market', 'High__Pre_market',
       'High__mkt_10_to_3_30', 'High__mkt_cl_3_30_to_4',
       'High__mkt_op_9_30_to_9_35', 'High__mkt_op_9_35_to_9_40',
       'High__mkt_op_9_40_to_9_45', 'High__mkt_op_9_45_to_10', 'High__unknown',
       'Low__After_market', 'Low__Pre_market', 'Low__mkt_10_to_3_30',
       'Low__mkt_cl_3_30_to_4', 'Low__mkt_op_9_30_to_9_35',
       'Low__mkt_op_9_35_to_9_40', 'Low__mkt_op_9_40_to_9_45',
       'Low__mkt_op_9_45_to_10', 'Low__unknown', 'Volume__After_market',
       'Volume__Pre_market', 'Volume__mkt_10_to_3_30',
       'Volume__mkt_cl_3_30_to_4', 'Volume__mkt_op_9_30_to_9_35',
       'Volume__mkt_op_9_35_to_9_40', 'Volume__mkt_op_9_40_to_9_45',
       'Volume__mkt_op_9_45_to_10', 'Volume__unknown',
       'time_cat_avg_price__After_market', 'time_cat_avg_price__Pre_market',
       'time_cat_avg_price__mkt_10_to_3_30',
       'time_cat_avg_price__mkt_cl_3_30_to_4',
       'time_cat_avg_price__mkt_op_9_30_to_9_35',
 

In [29]:
stocks_d_hlv_cat_pivot[['Date', 'Ticker', 
                        'High__mkt_op_9_30_to_9_35', 'High__mkt_op_9_35_to_9_40',
                        'High__mkt_op_9_40_to_9_45', 'High__mkt_op_9_45_to_10',
                        'High__mkt_10_to_3_30', 'High__mkt_cl_3_30_to_4', 
                        
                        'Low__mkt_op_9_30_to_9_35', 'Low__mkt_op_9_35_to_9_40',
                        'Low__mkt_op_9_40_to_9_45', 'Low__mkt_op_9_45_to_10',
                        'Low__mkt_10_to_3_30', 'Low__mkt_cl_3_30_to_4'
                      ]].to_pickle(f'{PROJ_PATH}{PROCESSING_TEMP_FOLDER_PATH}stocks_intraday_timecats_research.pkl') 


In [30]:
stocks_d_hlv_cat_pivot = stocks_d_hlv_cat_pivot[['Date', 'Ticker', 
                        
                        'High__Pre_market', 'Low__Pre_market', 
                        'Volume__Pre_market', 'time_cat_avg_price__Pre_market', 
                    
                        'High__mkt_op_9_30_to_9_35', 'High__mkt_op_9_35_to_9_40',
                        'High__mkt_op_9_40_to_9_45', 'High__mkt_op_9_45_to_10',
                        'High__mkt_cl_3_30_to_4', 
                        
                        'Low__mkt_op_9_30_to_9_35', 'Low__mkt_op_9_35_to_9_40',
                        'Low__mkt_op_9_40_to_9_45', 'Low__mkt_op_9_45_to_10',
                        'Low__mkt_cl_3_30_to_4',
                       
                        
                       'Volume__mkt_op_9_30_to_9_35', 'Volume__mkt_op_9_35_to_9_40',
                       'Volume__mkt_op_9_40_to_9_45', 'Volume__mkt_op_9_45_to_10',
                       'Volume__mkt_10_to_3_30', 'Volume__mkt_cl_3_30_to_4',
                       
                       'time_cat_avg_price__mkt_op_9_30_to_9_35', 'time_cat_avg_price__mkt_op_9_35_to_9_40',
                       'time_cat_avg_price__mkt_op_9_40_to_9_45', 'time_cat_avg_price__mkt_op_9_45_to_10', 
                       'time_cat_avg_price__mkt_10_to_3_30', 'time_cat_avg_price__mkt_cl_3_30_to_4',
                       
                       'High__After_market', 'Low__After_market',
                       'Volume__After_market', 'time_cat_avg_price__After_market'
                        
                       ]]

In [31]:
stocks_d_hlv_cat_pivot['High__mkt_op_15min'] = stocks_d_hlv_cat_pivot.apply(lambda row: max(row['High__mkt_op_9_30_to_9_35'], row['High__mkt_op_9_35_to_9_40'], row['High__mkt_op_9_40_to_9_45']) , axis=1)
stocks_d_hlv_cat_pivot['Low__mkt_op_15min'] = stocks_d_hlv_cat_pivot.apply(lambda row: min(row['High__mkt_op_9_30_to_9_35'], row['High__mkt_op_9_35_to_9_40'], row['High__mkt_op_9_40_to_9_45']) , axis=1)

stocks_d_hlv_cat_pivot['High__mkt_op_30min'] = stocks_d_hlv_cat_pivot.apply(lambda row: max(row['High__mkt_op_15min'], row['High__mkt_op_9_45_to_10'] ) , axis=1)
stocks_d_hlv_cat_pivot['Low__mkt_op_30min'] = stocks_d_hlv_cat_pivot.apply(lambda row: min(row['High__mkt_op_15min'], row['High__mkt_op_9_45_to_10'] ) , axis=1)


stocks_d_hlv_cat_pivot['Volume__mkt_op_15min'] = stocks_d_hlv_cat_pivot['Volume__mkt_op_9_30_to_9_35'] + \
                                                 stocks_d_hlv_cat_pivot['Volume__mkt_op_9_35_to_9_40'] + \
                                                 stocks_d_hlv_cat_pivot['Volume__mkt_op_9_40_to_9_45'] 

stocks_d_hlv_cat_pivot.tail(2)


,Date,Ticker,High__Pre_market,Low__Pre_market,Volume__Pre_market,time_cat_avg_price__Pre_market,High__mkt_op_9_30_to_9_35,High__mkt_op_9_35_to_9_40,High__mkt_op_9_40_to_9_45,High__mkt_op_9_45_to_10,High__mkt_cl_3_30_to_4,Low__mkt_op_9_30_to_9_35,Low__mkt_op_9_35_to_9_40,Low__mkt_op_9_40_to_9_45,Low__mkt_op_9_45_to_10,Low__mkt_cl_3_30_to_4,Volume__mkt_op_9_30_to_9_35,Volume__mkt_op_9_35_to_9_40,Volume__mkt_op_9_40_to_9_45,Volume__mkt_op_9_45_to_10,Volume__mkt_10_to_3_30,Volume__mkt_cl_3_30_to_4,time_cat_avg_price__mkt_op_9_30_to_9_35,time_cat_avg_price__mkt_op_9_35_to_9_40,time_cat_avg_price__mkt_op_9_40_to_9_45,time_cat_avg_price__mkt_op_9_45_to_10,time_cat_avg_price__mkt_10_to_3_30,time_cat_avg_price__mkt_cl_3_30_to_4,High__After_market,Low__After_market,Volume__After_market,time_cat_avg_price__After_market,High__mkt_op_15min,Low__mkt_op_15min,High__mkt_op_30min,Low__mkt_op_30min,Volume__mkt_op_15min
213123,2023-07-11,ZM,67.85,67.0000,0.0,67.593417,67.85,67.85,67.85,67.85,67.85,67.0000,67.0000,67.0000,67.0000,67.0000,0.0,0.0,0.0,0.0,0.0,0.0,67.593417,67.593417,67.593417,67.593417,67.593417,67.593417,67.85,67.0000,0.0,67.593417,67.85,67.85,67.85,67.85,0.0
213124,2023-07-11,ZTO,25.10,24.7315,0.0,25.098907,25.10,25.10,25.10,25.10,25.10,24.7315,24.7315,24.7315,24.7315,24.7315,0.0,0.0,0.0,0.0,0.0,0.0,25.098907,25.098907,25.098907,25.098907,25.098907,25.098907,25.10,24.7315,0.0,25.098907,25.10,25.10,25.10,25.10,0.0


In [32]:
stocks_d_hlv_cat_pivot.tail(2)

,Date,Ticker,High__Pre_market,Low__Pre_market,Volume__Pre_market,time_cat_avg_price__Pre_market,High__mkt_op_9_30_to_9_35,High__mkt_op_9_35_to_9_40,High__mkt_op_9_40_to_9_45,High__mkt_op_9_45_to_10,High__mkt_cl_3_30_to_4,Low__mkt_op_9_30_to_9_35,Low__mkt_op_9_35_to_9_40,Low__mkt_op_9_40_to_9_45,Low__mkt_op_9_45_to_10,Low__mkt_cl_3_30_to_4,Volume__mkt_op_9_30_to_9_35,Volume__mkt_op_9_35_to_9_40,Volume__mkt_op_9_40_to_9_45,Volume__mkt_op_9_45_to_10,Volume__mkt_10_to_3_30,Volume__mkt_cl_3_30_to_4,time_cat_avg_price__mkt_op_9_30_to_9_35,time_cat_avg_price__mkt_op_9_35_to_9_40,time_cat_avg_price__mkt_op_9_40_to_9_45,time_cat_avg_price__mkt_op_9_45_to_10,time_cat_avg_price__mkt_10_to_3_30,time_cat_avg_price__mkt_cl_3_30_to_4,High__After_market,Low__After_market,Volume__After_market,time_cat_avg_price__After_market,High__mkt_op_15min,Low__mkt_op_15min,High__mkt_op_30min,Low__mkt_op_30min,Volume__mkt_op_15min
213123,2023-07-11,ZM,67.85,67.0000,0.0,67.593417,67.85,67.85,67.85,67.85,67.85,67.0000,67.0000,67.0000,67.0000,67.0000,0.0,0.0,0.0,0.0,0.0,0.0,67.593417,67.593417,67.593417,67.593417,67.593417,67.593417,67.85,67.0000,0.0,67.593417,67.85,67.85,67.85,67.85,0.0
213124,2023-07-11,ZTO,25.10,24.7315,0.0,25.098907,25.10,25.10,25.10,25.10,25.10,24.7315,24.7315,24.7315,24.7315,24.7315,0.0,0.0,0.0,0.0,0.0,0.0,25.098907,25.098907,25.098907,25.098907,25.098907,25.098907,25.10,24.7315,0.0,25.098907,25.10,25.10,25.10,25.10,0.0


In [33]:
stocks_d_hlv_cat_pivot = stocks_d_hlv_cat_pivot[['Date', 'Ticker',  'High__Pre_market', 'Low__Pre_market',
                                                   'Volume__Pre_market', 'time_cat_avg_price__Pre_market',

                                                   'High__mkt_op_9_30_to_9_35', 'High__mkt_op_15min',  'High__mkt_op_30min',
                                                   'Low__mkt_op_9_30_to_9_35',  'Low__mkt_op_15min',  'Low__mkt_op_30min',
                                                   'High__mkt_cl_3_30_to_4', 'Low__mkt_cl_3_30_to_4',
                                                 
                                                   'Volume__mkt_op_15min', 'Volume__mkt_cl_3_30_to_4', 
                                                 
                                                   'High__After_market', 'Low__After_market', 'Volume__After_market',
                                                   'time_cat_avg_price__After_market' ]]

stocks_d_hlv_cat_pivot.tail(2) 

,Date,Ticker,High__Pre_market,Low__Pre_market,Volume__Pre_market,time_cat_avg_price__Pre_market,High__mkt_op_9_30_to_9_35,High__mkt_op_15min,High__mkt_op_30min,Low__mkt_op_9_30_to_9_35,Low__mkt_op_15min,Low__mkt_op_30min,High__mkt_cl_3_30_to_4,Low__mkt_cl_3_30_to_4,Volume__mkt_op_15min,Volume__mkt_cl_3_30_to_4,High__After_market,Low__After_market,Volume__After_market,time_cat_avg_price__After_market
213123,2023-07-11,ZM,67.85,67.0000,0.0,67.593417,67.85,67.85,67.85,67.0000,67.85,67.85,67.85,67.0000,0.0,0.0,67.85,67.0000,0.0,67.593417
213124,2023-07-11,ZTO,25.10,24.7315,0.0,25.098907,25.10,25.10,25.10,24.7315,25.10,25.10,25.10,24.7315,0.0,0.0,25.10,24.7315,0.0,25.098907


In [34]:
stocks_d_hlv_cat_pivot.columns = ['Date', 'Ticker', 'High__Pre_market', 'Low__Pre_market',
                                   'Volume__Pre_market', 'Avg_price__Pre_market',
                                   'High__mkt_op_5min', 'High__mkt_op_15min', 'High__mkt_op_30min',
                                   'Low__mkt_op_5min', 'Low__mkt_op_15min', 'Low__mkt_op_30min',
                                   'High__mkt_cl_30min', 'Low__mkt_cl_30min',
                                   'Volume__mkt_op_15min', 'Volume__mkt_cl_30min',
                                   'High__After_market', 'Low__After_market', 'Volume__After_market',
                                   'Avg_price__After_market'] 

In [35]:
print(stocks_d_hlv_cat_pivot.shape)
stocks_d_hlv_cat_pivot.tail(2) 

(213125, 20)


,Date,Ticker,High__Pre_market,Low__Pre_market,Volume__Pre_market,Avg_price__Pre_market,High__mkt_op_5min,High__mkt_op_15min,High__mkt_op_30min,Low__mkt_op_5min,Low__mkt_op_15min,Low__mkt_op_30min,High__mkt_cl_30min,Low__mkt_cl_30min,Volume__mkt_op_15min,Volume__mkt_cl_30min,High__After_market,Low__After_market,Volume__After_market,Avg_price__After_market
213123,2023-07-11,ZM,67.85,67.0000,0.0,67.593417,67.85,67.85,67.85,67.0000,67.85,67.85,67.85,67.0000,0.0,0.0,67.85,67.0000,0.0,67.593417
213124,2023-07-11,ZTO,25.10,24.7315,0.0,25.098907,25.10,25.10,25.10,24.7315,25.10,25.10,25.10,24.7315,0.0,0.0,25.10,24.7315,0.0,25.098907


In [36]:
stocks_d_hlv_cat['time_cat'].unique() 

array(['Pre_market', 'mkt_op_9_30_to_9_35', 'mkt_op_9_35_to_9_40',
       'mkt_op_9_40_to_9_45', 'mkt_op_9_45_to_10', 'mkt_10_to_3_30',
       'mkt_cl_3_30_to_4', 'After_market', 'unknown'], dtype=object)

### Market high-low daily

In [37]:
stocks_d_hlv_cat_mkt = stocks_d_hlv_cat[stocks_d_hlv_cat['time_cat'].isin(['mkt_op_9_30_to_9_35', 'mkt_op_9_35_to_9_40',
                                                                           'mkt_op_9_40_to_9_45', 'mkt_op_9_45_to_10', 'mkt_10_to_3_30',
                                                                           'mkt_cl_3_30_to_4'])].reset_index(drop=True)
print(stocks_d_hlv_cat_mkt.shape)
stocks_d_hlv_cat_mkt.tail(2) 

(1278750, 8)


,Date,Ticker,time_cat,High,Low,Volume,time_cat_avg_price,time_cat_seq
1278748,2023-07-11,ZTO,mkt_10_to_3_30,25.1,24.7315,0.0,25.098907,6.0
1278749,2023-07-11,ZTO,mkt_cl_3_30_to_4,25.1,24.7315,0.0,25.098907,7.0


In [38]:
stocks_d_hl_mkt = stocks_d_hlv_cat_mkt.groupby(['Ticker','Date']).agg({
    "High" : "max",
    "Low" :  "min"
}).reset_index()
stocks_d_hl_mkt.columns = ['Ticker','Date','mkt_day_high','mkt_day_low']
print(stocks_d_hl_mkt.shape)
stocks_d_hl_mkt.tail(2) 

(213125, 4)


,Ticker,Date,mkt_day_high,mkt_day_low
213123,ZTO,2023-07-10,25.13,24.6900
213124,ZTO,2023-07-11,25.10,24.7315


### minute level only market hours df

In [39]:
stocks_d_ohlcv_mkt_minute = stocks_daily_ohlcv[stocks_daily_ohlcv['time_cat'].isin(['mkt_op_9_30_to_9_35', 'mkt_op_9_35_to_9_40',
                                                                           'mkt_op_9_40_to_9_45', 'mkt_op_9_45_to_10', 'mkt_10_to_3_30',
                                                                           'mkt_cl_3_30_to_4'])].reset_index(drop=True)

print(stocks_d_ohlcv_mkt_minute.shape)
stocks_d_ohlcv_mkt_minute = pd.merge(stocks_d_ohlcv_mkt_minute, stocks_d_hl_mkt,
                             how='left',on=['Ticker','Date'])
print(stocks_d_ohlcv_mkt_minute.shape)
stocks_d_ohlcv_mkt_minute.tail(2) 


(66438434, 13)
(66438434, 15)


,Datetime,Open,High,Low,Close,Volume,Ticker,Value,Date,time,time_cat,avg_price,vol_price,mkt_day_high,mkt_day_low
66438432,2023-07-10 15:58:00-04:00,25.09,25.10,25.09,25.1,12848.0,ZTO,NaN,2023-07-10,15:58:00,mkt_cl_3_30_to_4,25.0950,322420.560,25.13,24.69
66438433,2023-07-10 15:59:00-04:00,25.10,25.12,25.09,25.1,116982.0,ZTO,NaN,2023-07-10,15:59:00,mkt_cl_3_30_to_4,25.1025,2936540.655,25.13,24.69


### Generating high_low df

In [40]:
stocks_d_ohlcv_mkt_HL = stocks_d_ohlcv_mkt_minute[( 
                                            (stocks_d_ohlcv_mkt_minute['High']==stocks_d_ohlcv_mkt_minute['mkt_day_high']) |
                                            (stocks_d_ohlcv_mkt_minute['Low']==stocks_d_ohlcv_mkt_minute['mkt_day_low'])
                                           )
                                          ].reset_index(drop=True)
print(stocks_d_ohlcv_mkt_HL.shape) 
stocks_d_ohlcv_mkt_HL.tail(2) 


(911215, 15)


,Datetime,Open,High,Low,Close,Volume,Ticker,Value,Date,time,time_cat,avg_price,vol_price,mkt_day_high,mkt_day_low
911213,2023-07-10 09:30:00-04:00,24.76,24.88,24.690,24.840,10150.0,ZTO,NaN,2023-07-10,09:30:00,mkt_op_9_30_to_9_35,24.7925,251643.8750,25.13,24.69
911214,2023-07-10 14:48:00-04:00,25.10,25.13,25.095,25.105,13449.0,ZTO,NaN,2023-07-10,14:48:00,mkt_10_to_3_30,25.1075,337670.7675,25.13,24.69


In [41]:
stocks_d_ohlcv_mkt_HL['High_true'] = stocks_d_ohlcv_mkt_HL['High']==stocks_d_ohlcv_mkt_HL['mkt_day_high']
stocks_d_ohlcv_mkt_HL['low_true'] = stocks_d_ohlcv_mkt_HL['Low']==stocks_d_ohlcv_mkt_HL['mkt_day_low'] 
stocks_d_ohlcv_mkt_HL['time'] = stocks_d_ohlcv_mkt_HL['Datetime'].dt.time


In [42]:
stocks_d_ohlcv_mkt_HL.tail(2) 


,Datetime,Open,High,Low,Close,Volume,Ticker,Value,Date,time,time_cat,avg_price,vol_price,mkt_day_high,mkt_day_low,High_true,low_true
911213,2023-07-10 09:30:00-04:00,24.76,24.88,24.690,24.840,10150.0,ZTO,NaN,2023-07-10,09:30:00,mkt_op_9_30_to_9_35,24.7925,251643.8750,25.13,24.69,False,True
911214,2023-07-10 14:48:00-04:00,25.10,25.13,25.095,25.105,13449.0,ZTO,NaN,2023-07-10,14:48:00,mkt_10_to_3_30,25.1075,337670.7675,25.13,24.69,True,False


In [43]:
stocks_d_hightime = stocks_d_ohlcv_mkt_HL[stocks_d_ohlcv_mkt_HL['High_true']==True].groupby(['Ticker','Date']).agg({
    'time' : "min"
}).reset_index()
stocks_d_hightime.columns = ['Ticker','Date','high_time']
print(stocks_d_hightime.shape)
stocks_d_hightime.tail(2) 


(207502, 3)


,Ticker,Date,high_time
207500,ZTO,2023-07-07,09:47:00
207501,ZTO,2023-07-10,14:48:00


In [44]:
stocks_d_lowtime = stocks_d_ohlcv_mkt_HL[stocks_d_ohlcv_mkt_HL['low_true']==True].groupby(['Ticker','Date']).agg({
    'time' : "min"
}).reset_index()
stocks_d_lowtime.columns = ['Ticker','Date','low_time']
print(stocks_d_lowtime.shape)
stocks_d_lowtime.tail(2) 


(207694, 3)


,Ticker,Date,low_time
207692,ZTO,2023-07-07,15:59:00
207693,ZTO,2023-07-10,09:30:00


In [45]:
stocks_d_HL_time = pd.merge(stocks_d_hightime, stocks_d_lowtime,
                                      how='left', on=['Ticker','Date'])
print(stocks_d_HL_time.shape)
stocks_d_HL_time.tail(2) 


(207502, 4)


,Ticker,Date,high_time,low_time
207500,ZTO,2023-07-07,09:47:00,15:59:00
207501,ZTO,2023-07-10,14:48:00,09:30:00


In [46]:
def get_time_diff_in_minutes(x_time, y_time):
    y = timedelta(hours=y_time.hour, minutes=y_time.minute)
    x = timedelta(hours=x_time.hour, minutes=x_time.minute)
    return np.abs(y - x).total_seconds() / 60 

In [47]:
print(stocks_d_HL_time.shape)
stocks_d_HL_time.head(2)

(207502, 4)


,Ticker,Date,high_time,low_time
0,AA,2022-06-06,14:19:00,11:20:00
1,AA,2022-06-07,15:59:00,09:32:00


In [48]:

for col in ['high_time', 'low_time']:
    stocks_d_HL_time[col] = stocks_d_HL_time.groupby(['Ticker'])[col].transform(lambda x: x.ffill() )
    stocks_d_HL_time[col] = stocks_d_HL_time.groupby(['Ticker'])[col].transform(lambda x: x.bfill() )


In [49]:
stocks_d_HL_time['HL_or_LH'] = np.where(stocks_d_HL_time['high_time'] < stocks_d_HL_time['low_time'],
                                                  "High_Low",
                                                  "Low_High"
                                                 )
#stocks_d_HL_time['HL_time_diff'] = stocks_d_HL_time['high_time'] - stocks_d_HL_time['low_time']

stocks_d_HL_time['HL_time_diff'] = stocks_d_HL_time.apply(lambda row: get_time_diff_in_minutes( row['high_time'], 
                                                                                               row['low_time']) , axis=1) 

print(stocks_d_HL_time.shape)
stocks_d_HL_time.tail() 


(207502, 6)


,Ticker,Date,high_time,low_time,HL_or_LH,HL_time_diff
207497,ZTO,2023-07-03,09:31:00,11:36:00,High_Low,125.0
207498,ZTO,2023-07-05,15:56:00,09:56:00,Low_High,360.0
207499,ZTO,2023-07-06,10:16:00,09:30:00,Low_High,46.0
207500,ZTO,2023-07-07,09:47:00,15:59:00,High_Low,372.0
207501,ZTO,2023-07-10,14:48:00,09:30:00,Low_High,318.0


In [50]:
stocks_d_HL_time['first_hr_high'] = np.where(
    (stocks_d_HL_time['high_time']>=DT.time(9, 30)) & (stocks_d_HL_time['high_time']<DT.time(10, 30)),
    True,
    False )

stocks_d_HL_time['first_hr_low'] = np.where(
    (stocks_d_HL_time['low_time']>=DT.time(9, 30)) & (stocks_d_HL_time['low_time']<DT.time(10, 30)),
    True,
    False )

stocks_d_HL_time['last_hr_high'] = np.where(
    (stocks_d_HL_time['high_time']>=DT.time(3,0)) & (stocks_d_HL_time['high_time']<DT.time(4,0)),
    True,
    False )

stocks_d_HL_time['last_hr_low'] = np.where(
    (stocks_d_HL_time['low_time']>=DT.time(3,0)) & (stocks_d_HL_time['low_time']<DT.time(4,0)),
    True,
    False ) 


In [51]:
stocks_d_HL_time.head(2)

,Ticker,Date,high_time,low_time,HL_or_LH,HL_time_diff,first_hr_high,first_hr_low,last_hr_high,last_hr_low
0,AA,2022-06-06,14:19:00,11:20:00,Low_High,179.0,False,False,False,False
1,AA,2022-06-07,15:59:00,09:32:00,Low_High,387.0,False,True,False,False


In [52]:
def get_minutes_from_mktopen(y_time):
    y = timedelta(hours=y_time.hour, minutes=y_time.minute)
    x = timedelta(hours=9, minutes=30)
    return (y - x).total_seconds() / 60 


In [53]:
stocks_d_HL_time['high_time'] = stocks_d_HL_time['high_time'].apply(lambda x: get_minutes_from_mktopen(x)) 
stocks_d_HL_time['low_time'] = stocks_d_HL_time['low_time'].apply(lambda x: get_minutes_from_mktopen(x)) 


In [54]:
print(stocks_d_hlv_cat_pivot.shape)
print(stocks_d_HL_time.shape)

(213125, 20)
(207502, 10)


In [55]:
intraday_final = pd.merge(stocks_d_hlv_cat_pivot, stocks_d_HL_time, how='left', on=['Ticker','Date'])
print(intraday_final.shape) 
intraday_final.head(2) 

(213125, 28)


,Date,Ticker,High__Pre_market,Low__Pre_market,Volume__Pre_market,Avg_price__Pre_market,High__mkt_op_5min,High__mkt_op_15min,High__mkt_op_30min,Low__mkt_op_5min,Low__mkt_op_15min,Low__mkt_op_30min,High__mkt_cl_30min,Low__mkt_cl_30min,Volume__mkt_op_15min,Volume__mkt_cl_30min,High__After_market,Low__After_market,Volume__After_market,Avg_price__After_market,high_time,low_time,HL_or_LH,HL_time_diff,first_hr_high,first_hr_low,last_hr_high,last_hr_low
0,2022-06-06,AA,64.25,62.40,0.0,63.313121,64.25,64.25,64.25,62.40,64.25,64.25,64.08,63.39,0.0,661978.0,64.10,63.80,151955.0,64.018133,289.0,110.0,Low_High,179.0,False,False,False,False
1,2022-06-06,AAL,16.45,16.12,0.0,16.284401,16.45,16.45,16.45,16.12,16.45,16.45,16.34,16.26,0.0,2878430.0,16.33,16.27,223700.0,16.309010,276.0,180.0,Low_High,96.0,False,False,False,False


In [57]:
intraday_final.to_pickle(f'{PROJ_PATH}{PROCESSING_TEMP_FOLDER_PATH}stocks_intraday_Features.pkl') 
